In [1]:
import os
import sys
import csv
import re
import math
import pandas as pd
# import pdb

import librosa
import os
import soundfile as sf
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import matplotlib.style as ms
from tqdm import tqdm
import pickle

import IPython.display
import librosa.display
ms.use('seaborn-muted')
%matplotlib inline


In [11]:

iemocap_full_release_path = "/Users/grigorkeropyan/Desktop/YSU_thesis/small_data/IEMOCAP_full_release/"

iemocap_pre_processed_data_path = "/Users/grigorkeropyan/Desktop/YSU_thesis/small_data/pre_processed_data/iemocap/"

In [3]:
info_line = re.compile(r'\[.+\]\n', re.IGNORECASE)

start_times, end_times, wav_file_names, emotions, vals, acts, doms = [], [], [], [], [], [], []

for sess in range(1, 2):
    emo_evaluation_dir = iemocap_full_release_path + '/Session{}/dialog/EmoEvaluation/'.format(sess)
    evaluation_files = [l for l in os.listdir(emo_evaluation_dir) if 'Ses' in l]
    for file in evaluation_files:
        x = re.search("^Ses.*", file)
        if x == None:
            continue
        with open(emo_evaluation_dir + file) as f:
            content = f.read()
        info_lines = re.findall(info_line, content)
        for line in info_lines[1:]:  # the first line is a header
            #pdb.set_trace()   
            start_end_time, wav_file_name, emotion, val_act_dom = line.strip().split('\t')
            start_time, end_time = start_end_time[1:-1].split('-')
            val, act, dom = val_act_dom[1:-1].split(',')
            val, act, dom = float(val), float(act), float(dom)
            start_time, end_time = float(start_time), float(end_time)
            start_times.append(start_time)
            end_times.append(end_time)
            wav_file_names.append(wav_file_name)
            emotions.append(emotion)
            vals.append(val)
            acts.append(act)
            doms.append(dom)

In [4]:

df_iemocap = pd.DataFrame(columns=['start_time', 'end_time', 'wav_file', 'emotion', 'val', 'act', 'dom'])

df_iemocap['start_time'] = start_times
df_iemocap['end_time'] = end_times
df_iemocap['wav_file'] = wav_file_names
df_iemocap['emotion'] = emotions
df_iemocap['val'] = vals
df_iemocap['act'] = acts
df_iemocap['dom'] = doms

In [5]:
df_iemocap

,start_time,end_time,wav_file,emotion,val,act,dom
0,6.7700,8.4600,Ses01M_impro01_F000,ang,1.5000,3.5000,4.5000
1,8.5500,10.6000,Ses01M_impro01_F001,ang,2.5000,3.5000,3.5000
2,14.4600,18.2300,Ses01M_impro01_F002,xxx,3.0000,3.0000,3.5000
3,19.4800,23.4300,Ses01M_impro01_F003,xxx,3.0000,3.0000,3.0000
4,23.4300,26.4675,Ses01M_impro01_F004,fru,2.5000,3.5000,3.5000
...,...,...,...,...,...,...,...
1814,394.4000,397.0800,Ses01M_script01_1_M038,sad,2.0000,2.6667,2.6667
1815,405.1800,407.5100,Ses01M_script01_1_M039,xxx,2.6667,2.0000,1.6667
1816,408.0000,410.1525,Ses01M_script01_1_M040,sad,2.6667,2.3333,3.0000
1817,412.1300,414.7800,Ses01M_script01_1_M041,ang,2.6667,3.0000,3.6667


In [7]:
sr = 44100
audio_vectors = {}

for sess in [1]:  # using one session due to memory constraint, can replace [5] with range(1, 6)
    wav_file_path = '{}/Session{}/dialog/wav/'.format(iemocap_full_release_path, sess)
    #pdb.set_trace()
    orig_wav_files = os.listdir(wav_file_path)
    
#     print(orig_wav_files)
    for orig_wav_file in tqdm(orig_wav_files):
        #print("Wav file is ", orig_wav_file)
        x = re.search("^Ses.*", orig_wav_file)
        if '.DS_Store' == orig_wav_file:
            continue
        if x == None:
            print("Skiping file", orig_wav_file)
            continue
#         print(wav_file_path + orig_wav_file)
        try:
            orig_wav_vector, _sr = librosa.load(wav_file_path + orig_wav_file, sr=sr)

            orig_wav_file, file_format = orig_wav_file.split('.')
            print("Working on file - ", orig_wav_file)
            #pdb.set_trace()
            for index, row in df_iemocap[df_iemocap['wav_file'].str.contains(orig_wav_file)].iterrows():
                start_time, end_time, truncated_wav_file_name, emotion, val, act, dom = row['start_time'], row['end_time'], row['wav_file'], row['emotion'], row['val'], row['act'], row['dom']
                start_frame = math.floor(start_time * sr)
                end_frame = math.floor(end_time * sr)
                truncated_wav_vector = orig_wav_vector[start_frame:end_frame + 1]


                X = librosa.stft(truncated_wav_vector)
                Xdb = librosa.amplitude_to_db(abs(X))
                plt.figure(figsize=(14, 5))
                #plt.figure()
                librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')
                plt.colorbar()

                #pdb.set_trace()
                plt.savefig(iemocap_pre_processed_data_path + "images/" + str(emotion) +"_"+ str(val) +"_"+ str(act) +"_"+ str(dom) + '.png')
                plt.close('all')
                audio_vectors[truncated_wav_file_name] = truncated_wav_vector

        except:
            print('An exception occured for {}'.format(orig_wav_file))
    with open(iemocap_pre_processed_data_path + '/audio_vectors_exp_{}.pkl'.format(sess), 'wb') as f:
        pickle.dump(audio_vectors, f)

  0%|          | 0/3 [00:00<?, ?it/s]

Working on file -  Ses01F_impro01


 67%|██████▋   | 2/3 [00:16<00:08,  8.02s/it]

Working on file -  Ses01M_script02_1


100%|██████████| 3/3 [01:02<00:00, 20.95s/it]


In [8]:
with open("../small_data/pre_processed_data/audio_vectors_exp_1.pkl", 'rb') as file:
    d = pickle.load(file)

In [ ]:
len(d[list(d.keys())[-3]])

In [ ]:
df_iemocap

In [9]:
len(d)

102